## 음악 타임 머신 프로젝트 
- 빌보드 에서 HOT 100에서 원하는 과거의 100곡을 스크래핑을 통해 얻기 
- 모든 노래 제목을 추출한 뒤 Spotify API를 통해 해당 일자의 플레이리스트를 생성 
- 그리고 Soptify에서 직접 노래를 확인하고 새로운 플레이리스트에 노래 추가! 

### 1. Billboard Hot 100 스크래핑 하기 
- 새 프로젝트 생성 
- 음악 여행을 몇 년도로 갈지 묻는 input() 프롬프트 만들기 -> 입력은 YYYY-MM-DD 형식으로 받기 
    - 예) Which year do you want to travel to? Type the date in this format YYYY-MM-DD:
- BeautifulSoup에 대해 배운 것을 토대로 해당 날짜의 탑 100 노래 제목을 파이썬 리스트로 스크래핑하기 

In [4]:
import requests
from bs4 import BeautifulSoup

In [3]:
# input 프롬프트 만들기 
question = input("Which year do you want to travel to? Type the date in this format YYYY-MM-DD:")
question

Which year do you want to travel to? Type the date in this format YYYY-MM-DD:2012-11-03


'2012-11-03'

In [37]:
# 내가 원하는 날짜를 기준으로 빌보드 Hot 100에 있는 제목 스크래핑 하기 
url = f"https://www.billboard.com/charts/hot-100/{question}/"
response = requests.get(url)
bilboard_page = response.text

soup = BeautifulSoup(bilboard_page, "html.parser")
music_title_spans = soup.select("li > ul > li > h3")
music_title = [title.getText().replace("\n", "").replace("\t", "") for title in music_title_spans]
music_title

['One More Night',
 'Gangnam Style',
 'Some Nights',
 'We Are Never Ever Getting Back Together',
 'Die Young',
 'As Long As You Love Me',
 'Too Close',
 'Diamonds',
 'Let Me Love You (Until You Learn To Love Yourself)',
 'Blow Me (One Last Kiss)',
 "Don't Wake Me Up",
 'Good Time',
 'State Of Grace',
 'Lights',
 'Locked Out Of Heaven',
 'Everybody Talks',
 'Home',
 'Clique',
 'Whistle',
 '50 Ways To Say Goodbye',
 'Blown Away',
 'Adorn',
 'I Knew You Were Trouble.',
 'I Cry',
 'Call Me Maybe',
 'Somebody That I Used To Know',
 'Ho Hey',
 'Wide Awake',
 'Cruise',
 'Titanium',
 "Live While We're Young",
 'Payphone',
 'Mercy',
 'I Will Wait',
 "It's Time",
 'Finally Found You',
 'Pop That',
 'Wanted',
 'Give Your Heart A Break',
 'Hard To Love',
 'The A Team',
 'Kiss Tomorrow Goodbye',
 'No Lie',
 "I Won't Give Up",
 'Skyfall',
 'Bandz A Make Her Dance',
 'Girl On Fire',
 '2 Reasons',
 'Birthday Song',
 'Ball',
 'Pound The Alarm',
 'Thinkin Bout You',
 'Turn On The Lights',
 'Catch My Bre

### 2. Spotify를 통한 인증 
- 스포티파이 무료 계정 만들고, 개발자 대시보드로 가서 새로운 앱 만들기 
- 스포티파이로 인증하기는 꽤나 복잡하다. 그래서 프로젝트를 좀 더 쉽게 만들기 위해 만든 파이썬 스포티파이 모듈인 스포티파이를 사용 
    - pip install spotipy
- 스포티파이 앱을 만들었다면, Client ID와 Client Secret을 파이썬 프로젝트로 복사, 그리고 Client ID와 Client Secret을 사용해서 파이썬 프로젝트를 스포티파이로 인증하는 방법 찾기 
- 리다이렉트 URI로 http://example.com를 사용하기. 현재 사용자 ID(여러분의 스포티파이 사용자명)를 가져오려고 한다. 문서에 있는 대로 스포티파이 대시보드에서도 리다이렉트 URI를 반드시 설정하기
    - 힌트 1: 스포티파이 앱에 본인 Client ID와 Secret이 있어야 하며, 위 이미지에 있는 Client ID와 Secret으로는 동작하지 않음
    - 힌트 2: 필요한 메소드는 이 링크를 참고: https://spotipy.readthedocs.io/en/2.13.0/#spotipy.oauth2.SpotifyOAuth
    - 힌트 3: Client ID와 Secret은 export나 set을 사용하지 말고 바로 SpotifyOAuth() 생성자로 전달.
    - 힌트 4: 스포티파이에서 비공개 플레이리스트를 만들려면, ’playlist-modify-private’ 범위가 필요.
    - 힌트 5: 제대로 했다면, 동의 페이지가 나옴. (반드시 ‘동의’를 누르기)
- 그 다음 example.com 페이지가 나오며, 주소창에 있는 전체 URL을 복사. 프롬프트가 나오면 해당 URL 복사해서 넣기
- 인증된 사용자의 user id(여러분의 스포티파이 사용자 이름)를 가져오기
    - 힌트 1: 이 메소드가 필요: https://spotipy.readthedocs.io/en/2.13.0/#spotipy.client.Spotify.current_user

    - 힌트 2: 위 메소드는 딕셔너리를 반환하므로 ‘id’ key에 해당하는 값을 찾기

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [5]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="Your ID",                # 스포티파이 클라이언트 아이디
                                               client_secret="Your Secret key",    # 스포티파이 클라이언트 시크릿키
                                               redirect_uri="http://example.com",                   # 리다이렉션 URL 
                                               scope="user-library-read playlist-modify-public"))   # 비공개 플레이 리스트 만들기 
user_key = sp.current_user()["id"]                   # 인증으로 받은 유저 아이디 
play_result = sp.user_playlists(user=user_key)       # 플레이 리스트 현황 확인 
play_result

{'href': 'https://api.spotify.com/v1/users/31d4lredsunq5fh7w7djkp76e76i/playlists?offset=0&limit=50',
 'items': [],
 'limit': 50,
 'next': None,
 'offset': 0,
 'previous': None,
 'total': 0}